In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import re
from moviepy.editor import *

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_holistic = mp.solutions.holistic

df = pd.DataFrame()
frames = []

In [2]:
import os
path = os.getcwd()

In [3]:
# Function to adjust video duration, frame rate, width, and height
def adjust_video_duration(video_path, output_duration, output_fps, width, height):
    video_clip = VideoFileClip(video_path, target_resolution=(height, width))
    video_clip = video_clip.without_audio()
    
    duration = video_clip.duration
    # getting only first 5 seconds
    clip = video_clip.subclip(0, duration)
    
    # applying speed effect
    final = clip.fx(vfx.speedx, duration)
    return final

In [4]:
def extract_video(video, target, sequence_id, target_width, target_height):
    global frames
    added_rows = 0
    # For webcam input:
    cap = cv2.VideoCapture(video)
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the current duration of the video
    video_duration = frame_count / fps
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the BGR frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame with the hand tracking model
        results = hands.process(rgb_frame)
         
        if results.multi_hand_landmarks:
            row_data = {
                "sequence_id": sequence_id,
                "target": target
            }
            detected_hands = []
            for idx, landmarks in enumerate(results.multi_hand_landmarks):
                # Iterate through detected hand landmarks
                for landmark_idx, landmark in enumerate(landmarks.landmark):
                    x, y = landmark.x * frame.shape[1], landmark.y * frame.shape[0]
                    
                    # Determine hand side based on the index of the hand in the list
                    if idx == 0:
                        hand_side = "Left"
                    else:
                        hand_side = "Right"
                    
                    row_data[f'x_{hand_side}_hand_{landmark_idx}'] =  x / target_width
                    row_data[f'y_{hand_side}_hand_{landmark_idx}'] =  y / target_height
            
                    detected_hands.append(hand_side)

                    # Draw circles on the frame
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
                    cv2.putText(frame, hand_side, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            added_rows += 1
            frames.append(row_data)
        cv2.imshow('Hand Tracking', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        elif (added_rows == 30 and fps > 30):
            print('-' * 20)
            print("FPS: ", fps, " VIDEO DURATION: ", video_duration, "TARGET: ", target)
            print('-' * 20)
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
import glob, os
import time
os.chdir("D:\Documents\Trabajo\megaproyecto_model\data/asl_videos")

sequence_id = 0
for file in glob.glob("*.mp4"):
    sequence_id += 1
    name = " ".join(file.split(" ")[:-1])

    input_path = f'D:\Documents\Trabajo\megaproyecto_model\data/asl_videos/{file}'
    output_path = 'D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/adjusted_video.mp4'

    target_width = 640
    target_height = 360
    desired_fps = 30

    if (name == "no"):
        adjusted_video = adjust_video_duration(
            video_path=input_path,
            output_duration=1,
            output_fps=30,
            width=target_width,
            height=target_height
        )
        # Save the adjusted video to the output file
        adjusted_video.write_videofile(output_path, codec='libx264', verbose=False, logger=None)


        extract_video(output_path, name, sequence_id, target_width, target_height)

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(frames)

In [ ]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
dir_data = "D:\Documents\Trabajo\megaproyecto_model" + "/data.csv"
df.to_csv(dir_data, index=False)

In [ ]:
validation_path = "D:\Documents\Trabajo\megaproyecto_model" + "/validation.csv"

import random
grouped = df.groupby('target')
selected_validation_ids = []
for group_name, group_indices in grouped.groups.items():
    ids = list(set(df.loc[group_indices]['sequence_id'].values))
    if (len(ids) < 0):
        raise ("Error")
    selected_validation_ids.append(random.choice(ids))

validation_df = df[df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
validation_df.to_csv(validation_path, index=False)

In [ ]:
train_path = "D:\Documents\Trabajo\megaproyecto_model" + "/train.csv"
train_df = df[~df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
train_df.to_csv(train_path, index=False)